# Exercise 1: Optical Flow

https://docs.opencv.org/4.6.0/d4/dee/tutorial_optical_flow.html

In this exercise you will:
- Learn the concepts of optical flow and its estimation using the [Lucas-Kanade](https://en.wikipedia.org/wiki/Kanade%E2%80%93Lucas%E2%80%93Tomasi_feature_tracker) method.
- Use the OpenCV function [cv.goodFeaturesToTrack()](https://docs.opencv.org/4.6.0/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541) to detect *Shi-Tomasi* corner points in a frame.
- Use the OpenCV function [cv.calcOpticalFlowPyrLK()](https://docs.opencv.org/4.6.0/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323) to track the feature points in a video.
- Create a dense optical flow field using the OpenCV function [cv.calcOpticalFlowFarneback()](https://docs.opencv.org/4.6.0/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af).

As a first step, let's import the python modules we need.

In [1]:
import cv2 as cv
import numpy as np

## Optical Flow

<img width="400" height="150" src="../notebook_images/optical_flow_basic1.jpg" style="padding-left: 10px; float: right;">

Optical flow is the pattern of apparent motion of image objects between two consecutive frames, caused by the movemement of objects or the camera. An "optical flow" is a 2D vector field, where each vector is a displacement vector showing the movement of points from a first (previous) frame to a second (current) frame. For example, consider the image to the right. It shows a red ball moving in 5 consecutive frames, where the arrow shows its displacement vector.

 Optical flow has many applications in areas like:
- [Structure from Motion (SfM)](https://en.wikipedia.org/wiki/Structure_from_motion#:~:text=Structure%20from%20motion%20(SfM)%20is,computer%20vision%20and%20visual%20perception).
- Video Compression.
- Video Stabilization.
- Etc.

Optical flow works on several assumptions:

1. The pixel intensities of an object do not change between consecutive frames.
2. Neighbouring pixels have similar motion.

Consider a pixel $I(x,y,t)$ in the first frame (notice time $t$ is added as a third dimension). It moves a distance $(dx,dy)$ in the next frame, taken at time $t+dt$. So, since those two pixels (one in each frame) are the same pixel, and the pixel intensity doesn't change, we can express this as:

$$
I(x,y,t) = I(x+dx, y+dy, t+dt)
$$

We then approximate the right-hand side with a Taylor series expansion, remove common terms, and divide by $dt$ to get the following equation:

$$
f(x,y,t) = f(x+u, y+v, t+1)
$$

$$
f(x,y,t) \approx f(x, y, t) + uf_{x}(x, y, t) + vf_{y}(x, y, t) + f_{t}(x, y, t)
$$

$$
f_x u + f_y v + f_t = 0
$$

where:

$$
f_x = \frac{\partial f}{\partial x} \; ; \; f_y = \frac{\partial f}{\partial y}
$$

$$
u = \frac{dx}{dt} \; ; \; v = \frac{dy}{dt}
$$

The equation $f_x u + f_y v + f_t = 0$ is called the *Optical Flow Equation*. In it, we can find the image gradients $f_x$ and $f_y$. Similarly, $f_t$ is the gradient along the time dimension. But $u$ and $v$ are unknown. We can't solve a single equation with two unknowns, directly. Several methods exist to solve this problem, where one of them is the *Lucas-Kanade method*.

### Lucas-Kanade method

The Lucas-Kanade method assumes that all the neighbouring pixels will have a similar motion. It takes a 3x3 patch (window, $w$) around a point, where all the 9 points are assumed to have a similar motion. We can find $(f_x, f_y, f_t)$ for these 9 points. So, now our problem becomes solving 9 equations with two unknowns, which is over-determined. A better solution is obtained using a least square fit.

$$
E(u,v) = \sum_{i \in w}{[uf_{x_i}(x_i, y_i, t_i) + vf_{y_i}(x_i, y_i, t_i) + f_{t_i}(x_i, y_i, t_i)]}^2
$$

$$
\frac{\partial E(u,v)}{\partial u} = 2 \sum_{i \in w}{[uf_{x_i}^{2}(x_i, y_i, t_i) + vf_{y_i}(x_i, y_i, t_i)f_{x_i}(x_i, y_i, t_i) + f_{t_i}(x_i, y_i, t_i)f_{x_i}(x_i, y_i, t_i)]} = 0
$$

$$
\frac{\partial E(u,v)}{\partial v} = 2 \sum_{i \in w}{[uf_{x_i}(x_i, y_i, t_i)f_{y_i}(x_i, y_i, t_i) + vf_{y_i}^{2}(x_i, y_i, t_i) + f_{t_i}(x_i, y_i, t_i)f_{y_i}(x_i, y_i, t_i)]} = 0
$$

Below is the final solution, which involves solving two equations with two unknowns.

$$
\begin{bmatrix} u \\ v \end{bmatrix} =

\begin{bmatrix}
    \sum_{i}{f_{x_i}}^2  &  \sum_{i}{f_{x_i} f_{y_i} } \\
    \sum_{i}{f_{x_i} f_{y_i}} & \sum_{i}{f_{y_i}}^2
\end{bmatrix}^{-1}

\begin{bmatrix}
    - \sum_{i}{f_{x_i} f_{t_i}} \\
    - \sum_{i}{f_{y_i} f_{t_i}}
\end{bmatrix}
$$

<img width="180" height="180" src="../notebook_images/image_pyramid.jpg" style="padding-left:10px; float: right;">

Notice the similarity of the inverse matrix above with the matrix used in the *Harris Corner Detector*, i.e. it suggests that corners are good points to be tracked.

From a user's point-of-view, the idea is simple, i.e. we supply a set of points to track, and we obtain the optical flow vectors of those points. One problem with this approach, is that it will fail when there is large motions between frames. This can be solved using an [image pyramid](https://en.wikipedia.org/wiki/Pyramid_(image_processing)) (as shown in the figure to the right), where small motions are removed and large motions become small, as we *go up* in the pyramid (i.e. use a down-sampled version of the original image). See [this link](https://docs.opencv.org/4.6.0/d4/d1f/tutorial_pyramids.html) and [this link](https://pyimagesearch.com/2015/03/16/image-pyramids-with-python-and-opencv) for creating image pyramids in OpenCV. Applying the *Lucas-Kanade* method in such a higher pyramid layer, we get the optical flow, along with the scale at that pyramid layer.

### Lucas-Kanade Optical Flow in OpenCV

OpenCV provides all this functionality in a single function [cv.calcOpticalFlowPyrLK()](https://docs.opencv.org/4.6.0/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323), where "*PyrLK*" stands for "*Pyramid Lucas Kanade*". In the code example below, we create a simple application that tracks a set of points in a video. We select the initial set of points (*Shi-Tomasi* corner points) from the first frame using the OpenCV function [cv.goodFeaturesToTrack()](https://docs.opencv.org/4.6.0/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541). Then we iteratively track those points using the *Lucas-Kanade* optical flow method. In every iteration, we pass the previous frame, previous set of points, and next frame to the function [cv.calcOpticalFlowPyrLK()](https://docs.opencv.org/4.6.0/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323), which returns the next set of points, along with a status that has a value of 1 if a certain next point is found, else zero.

The code below doesn't check *how correct* the next keypoints are. So, even if a feature point disappears in a frame, there is a chance that the optical flow method will find a next point which looks close to the missing point. So, to obtain robust tracking, corner points should be detected in particular intervals. Therefore, OpenCV finds the feature points once every 5 frames, and also runs a backward-check of the optical flow points obtained, to only select *good points*.

In [2]:
# Load the video
cap = cv.VideoCapture('../data/slow.mp4')

# Define the parameters for Shi Tomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Define the parameters for Lucas Kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors (to visualize tracking for different keypoints)
color = np.random.randint(0, 255, (100, 3))

# Capture the first frame and find (Shi Tomasi) corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# Iterate for every frame ...
while(cap.isOpened()):
    
    # Capture the next frame
    ret, frame = cap.read()
    if ret is False:
        break        
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculate the optical flow with the Lucas Kanade method
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # Draw the tracks (i.e. the "tacking lines")
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)

    # Display the frame with tracking information
    cv.imshow('frame', img)
    if cv.waitKey(30) & 0xff == ord('q'):
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

# Release the VideoCapture and
# destroy any windows we have created
cap.release()
cv.destroyAllWindows()

## Dense Optical Flow in OpenCV

The *Lucas-Kanade* method computes the optical flow for a sparse feature set (in our example, the corners are detected using the *Shi-Tomasi* algorithm, very similar to the *Harris Corner Detection* algorithm). OpenCV provides another algorithm [cv.calcOpticalFlowFarneback()](https://docs.opencv.org/4.6.0/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af) to find the dense optical flow. It computes the optical flow for **all** the points in the frame. It is based on Gunner Farneback’s algorithm which is explained in [Two-Frame Motion Estimation Based on Polynomial Expansion](http://liu.diva-portal.org/smash/get/diva2%3A273847/FULLTEXT01.pdf) by Gunner Farneback in 2003.

The example code below, shows how to find the dense optical flow using the above algorithm. We obtain a 2-channel array with optical flow vectors $(u,v)$. Then we find their magnitude and direction. Finally, we color-code the result for better visualization, where **direction** corresponds to the **Hue** value of the image, and the **magnitude** corresponds to the **Value** plane.

In [3]:
# Load the video
cap = cv.VideoCapture("../data/vtest.avi")

# Capture the first frame
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

# Iterate for every frame ...
while(cap.isOpened()):

    # Capture the next frame
    ret, frame2 = cap.read()
    if ret is False:
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

    # Calculate the optical flow with the Farneback method
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Calculate the color-coding based on
    # the angle and magnitude of the flow vectors
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang * 180 / np.pi / 2
    hsv[...,2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    # Display the original frame
    cv.imshow('frame2', frame2)
    
    # Display the original frame with the color-coded tracking information
    cv.imshow('bgr', bgr)
    
    # Give the user the option to save frames to disk
    k = cv.waitKey(30) & 0xff
    if k == ord('q'):
        break
    elif k == ord('s'):
        cv.imwrite('../data/opticalfb.png', frame2)
        cv.imwrite('../data/opticalhsv.png', bgr)
    
    # Set the previous frame to the current frame
    prvs = next

# Release the VideoCapture and
# destroy any windows we have created
cap.release()
cv.destroyAllWindows()